In [16]:
import pandas as pd
import os

In [3]:
historic = pd.read_csv('data/historic/all_data.csv')

In [5]:
historic.head()

,DeliveryDate,DeliveryHour,DeliveryInterval,DSTFlag,SettlementPointName,SettlementPointType,SettlementPointPrice
0,01/01/2018,1,1,N,HB_BUSAVG,SH,27.37
1,01/01/2018,1,2,N,HB_BUSAVG,SH,27.37
2,01/01/2018,1,3,N,HB_BUSAVG,SH,27.15
3,01/01/2018,1,4,N,HB_BUSAVG,SH,27.13
4,01/01/2018,1,1,N,HB_HOUSTON,HU,27.68


In [15]:
savefolder = "data/live"
savepath = os.path.join(os.getcwd(), savefolder)

In [18]:
historicPoints = pd.read_csv(os.path.join(savepath, "all_data.csv").replace("live", "historic"))['SettlementPointName']

In [19]:
historicPoints

0          HB_BUSAVG
1          HB_BUSAVG
2          HB_BUSAVG
3          HB_BUSAVG
4         HB_HOUSTON
5         HB_HOUSTON
6         HB_HOUSTON
7         HB_HOUSTON
8          HB_HUBAVG
9          HB_HUBAVG
10         HB_HUBAVG
11         HB_HUBAVG
12          HB_NORTH
13          HB_NORTH
14          HB_NORTH
15          HB_NORTH
16          HB_SOUTH
17          HB_SOUTH
18          HB_SOUTH
19          HB_SOUTH
20           HB_WEST
21           HB_WEST
22           HB_WEST
23           HB_WEST
24            LZ_AEN
25            LZ_AEN
26            LZ_AEN
27            LZ_AEN
28            LZ_AEN
29            LZ_AEN
             ...    
581374        LZ_AEN
581375        LZ_AEN
581376        LZ_CPS
581377        LZ_CPS
581378        LZ_CPS
581379        LZ_CPS
581380    LZ_HOUSTON
581381    LZ_HOUSTON
581382    LZ_HOUSTON
581383    LZ_HOUSTON
581384       LZ_LCRA
581385       LZ_LCRA
581386       LZ_LCRA
581387       LZ_LCRA
581388      LZ_NORTH
581389      LZ_NORTH
581390      L

In [6]:
live = pd.read_csv('data/live/all_data.csv')

In [7]:
live.head()

,DeliveryDate,DeliveryHour,DeliveryInterval,SettlementPointName,SettlementPointType,SettlementPointPrice,DSTFlag
0,08/21/2018,8,3,AEEC,RN,20.61,N
1,08/21/2018,8,3,AMISTAD_ALL,RN,20.61,N
2,08/21/2018,8,3,AMOCOOIL_CC1,LCCRN,20.61,N
3,08/21/2018,8,3,AMOCOOIL_CC2,LCCRN,20.61,N
4,08/21/2018,8,3,AMOCO_PUN1,PUN,20.61,N
